# Libraries and helper functions

In [ ]:
import os
import tensorflow as tf
import keras
from keras.models import Sequential,load_model
from keras.utils.vis_utils import plot_model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd
import math
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score,confusion_matrix,roc_curve,auc
from sklearn.utils import shuffle 
from sklearn.metrics import log_loss,mean_squared_error

from resources.data_utils import split_dataset, DataGenerator
from resources.model_utils import build_model, train_model_noitr,evaluate_model
from resources.savers import TrainingSaver
from resources.utils import label_str_to_dec,prediction_standardized,aggregate_generator_labels,label_distribution

In [ ]:
# STACKED MODEL
def stacked_dataset(members, X_generator):        
    stackX = None 
    for model in members:
        #yhat = prediction_standardized(evaluate_model(model, evaluation_generator=X_generator, workers=6)) #one-hot
        yhat = evaluate_model(model, evaluation_generator=X_generator, workers=6)
        if stackX is None:
            stackX = yhat
        else:
            stackX = np.dstack((stackX, yhat))
            
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]),order='F')
    return stackX

def build_stacked_model(num_layer,num_models,num_classes,optimizer):
    model = Sequential()
    if num_layer == 0:
        model.add(Dense(num_classes,input_dim=num_models*num_classes,activation='softmax')) 
        
    elif num_layer == 1:
        model.add(Dense(40, input_dim=num_models*num_classes, activation='relu'))
        model.add(Dense(num_classes,input_dim=60,activation='softmax')) 
        
    elif num_layer == 2:
        model.add(Dense(60, input_dim=num_models*num_classes, activation='relu'))
        model.add(Dense(30, activation='relu'))   
        model.add(Dense(num_classes,input_dim=30,activation='softmax'))
        
    elif num_layer == 3:
        model.add(Dense(60, input_dim=num_models*num_classes, activation='relu'))
        model.add(Dense(45, activation='relu'))
        model.add(Dense(25, activation='relu'))
        model.add(Dense(num_classes,input_dim=25,activation='softmax'))
        
    model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
    return model

# fit stacked model based on the outputs from the ensemble members
def fit_stacked_model(model_stack,X_train_stacked,y_train,X_val_stacked,y_val,epochs,batch_size,early_stopping):
    history = model_stack.fit(X_train_stacked, \
                              y_train, \
                              validation_data=(X_val_stacked, y_val),\
                              epochs=epochs, \
                              batch_size=batch_size,\
                              callbacks=[early_stopping])
    return model_stack,history

def plot_EM_trian_history(history,path_out,IFPLOT=True):
    fig, axs = plt.subplots(1, 2,figsize=(10, 5))
    axs[0].plot(history.history['accuracy'])
    axs[0].plot(history.history['val_accuracy'])
    axs[0].set_title('Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].legend(['Train', 'Test'], loc='upper left')
    axs[1].plot(history.history['loss'])
    axs[1].plot(history.history['val_loss'])
    axs[1].set_title('Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].legend(['Train', 'Test'], loc='upper left')
    if IFPLOT:
        history_figname = path_out+'stacked_model_training_hisotry.png'
        plt.savefig(history_figname)

In [ ]:
# FOR ROC
def compute_roc(char_list,y_test,pred_prob):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(len(char_list)):
        fpr[i], tpr[i], _ = roc_curve(y_test[:,i], pred_prob[:,i]) 
        roc_auc[i] = auc(fpr[i], tpr[i])
    return fpr,tpr,roc_auc


def plot_roc(axs,label_list,fpr,tpr,roc_auc,title,addlegend=False):
    for i in range(len(char_list)):
        axs.plot(fpr[i], tpr[i], label='Class {0}'
                 ''.format(i, roc_auc[i]))
    axs.plot([0, 1], [0, 1], 'k--')
    axs.set_xlim([0.0, 1.0])
    axs.set_ylim([0.0, 1.05])
    axs.set_xlabel('False Positive Rate',fontsize=20,fontname='Calibri', color="black")
    axs.set_ylabel('True Positive Rate',fontsize=20,fontname='Calibri', color="black")
    axs.tick_params(axis='x', labelsize=20,color="black",labelcolor="black") 
    axs.tick_params(axis='y', labelsize=20,color="black",labelcolor="black")
    axs.set_title(title)
    if addlegend == True:
        axs.legend(loc="right",bbox_to_anchor=(1.3, 0.5))
        
def ave_roc(label_list,fpr,tpr,roc_auc):
    tpr_all = []
    roc_auc_all = []
    for i in range(len(label_list)):
        fpr_current = fpr[i]
        tpr_current = tpr[i]
        fpr_vals = np.linspace(0, 1, 201)
        tpr_current_interp = np.interp(fpr_vals, fpr_current, tpr_current)
        tpr_all.append(tpr_current_interp)
        roc_auc_current = roc_auc[i]
        roc_auc_all.append(roc_auc_current)
        
    ave_tpr = np.average(tpr_all, axis=0)
    std_tpr = np.std(tpr_all, axis=0)
    ave_roc_auc = np.average(roc_auc_all)
    return ave_tpr,std_tpr,ave_roc_auc

def plot_ave_roc(axs,ave_tpr,std_tpr,ave_roc_auc,title,addlegend=False):
    
    fpr = np.linspace(0, 1, 201)
    tprs_upper = np.minimum(ave_tpr + std_tpr, 1)
    tprs_lower = np.maximum(ave_tpr - std_tpr, 0)
    axs.plot(fpr, ave_tpr,linewidth=4)
    axs.fill_between(fpr, tprs_lower, tprs_upper, color='grey', alpha=.2)
    axs.set_xlim([0.0, 1.0])
    axs.set_ylim([0.0, 1.05])
    axs.set_xlabel('False Positive Rate',fontsize=20,fontname='Calibri', color="black")
    axs.set_ylabel('True Positive Rate',fontsize=20,fontname='Calibri', color="black")
    axs.tick_params(axis='x', labelsize=20,color="black",labelcolor="black") 
    axs.tick_params(axis='y', labelsize=20,color="black",labelcolor="black") 
    axs.set_title(title,fontsize=20,fontname='Calibri', color="black")
    if addlegend == True:
        axs.legend(loc="right",bbox_to_anchor=(2, 0.5))

# Load dataset and base models

In [ ]:
individual_train_size = 1500
split_dict = {12000:1,6000:2,3000:3,1500:4,750:5,375:6,150:7}
split_idx  = split_dict.get(individual_train_size)

dataset = 'dataset_name'
path = "/home/datasets/"+dataset+'/'
path_final = path + "preprocessed_80/"
path_bm = "/home/base_models/'
path_plaintext = path
filename = path_plaintext + 'labels_spot_binary.csv'

num_classes = 15 # to change according to the dictionary size
char_list = [str(i) for i in range(1, num_classes+1)]
char2int = {char: i for i, char in enumerate(char_list)}
num_replicates = 1000
dataset_size = num_classes * num_replicates
image_dimension = (80, 80) 

In [ ]:
num_base_model = 5 # number of base models
num_layer = 0 # depth of stacked model

# create output folder
path_out = path_bm + str(individual_train_size)+"_HL_"+str(num_layer)+"_BM_"+str(num_base_model)+"/"
if not os.path.exists(path_out):
    os.makedirs(path_out)

In [ ]:
# Rename models and txt files
for i in range(1,num_base_model+1):
    
    filename1 = path_bm+'class_15_trainsize_'+str(round(individual_train_size*(1-PORTION)))+'_opt_Adam_batch_size_4_test_'+str(i)+'_accuracy.txt'
    if os.path.exists(filename1):
        print('RENAMING')
        new_filename1 = path_bm+'model_'+str(i)+'_accuracy.txt'
        os.rename(filename1,new_filename1)
    else: print('NO NEED TO RENAME')
        
    filename2 = path_bm+'class_15_trainsize_'+str(round(individual_train_size*(1-PORTION)))+'_opt_Adam_batch_size_4_test_'+str(i)+'_model.h5'
    if os.path.exists(filename2):
        new_filename2 = path_bm+'model_'+str(i)+'.h5'
        os.rename(filename2,new_filename2)

In [ ]:
# Labels
labels = []
with open(filename, "r") as csvFile:
    for row in csvFile:
        labels.append(row[:-1])
labels = np.asarray(labels)
main_labels = label_str_to_dec(labels[0:dataset_size], char2int)

# Images
image_prefix = "FImg_ID_"
image_suffix = ".jpg"
images_str = [
    "{}{}{}{}".format(path_final, image_prefix, img_idx, image_suffix)
    for img_idx in range(1, dataset_size + 1)
]
main_dataset = pd.DataFrame({"img_path": images_str, "label": main_labels})

# Split dataset
generation_params = {"dim": image_dimension,\
                     "nb_classes": num_classes,\
                     "column_img": "img_path",\
                     "column_label": "label",}

df_train, df_valid, df_test = split_dataset(data_frame=main_dataset,\
                                            rank=split_idx,\
                                            column_label="label",\
                                            random_state=25)
train_generator = DataGenerator(data_frame=df_train, batch_size=25, shuffle=True, **generation_params)
valid_generator = DataGenerator(data_frame=df_valid, batch_size=100, shuffle=False, **generation_params)
test_generator  = DataGenerator(data_frame=df_test, batch_size=100, shuffle=False, **generation_params)

# Build ensemble model and the new training set

In [ ]:
# Load base models
members = []
for i in range(1, num_base_model+1):
    modeli = load_model(path_bm + "model_" + str(i) + ".h5")
    members.append(modeli) 

In [ ]:
# Make the stacked dataset
X_train_stacked = stacked_dataset(members, train_generator)
X_valid_stacked = stacked_dataset(members, valid_generator)
X_test_stacked  = stacked_dataset(members, test_generator)

y_train = aggregate_generator_labels(data_generator=train_generator)
y_valid = aggregate_generator_labels(data_generator=valid_generator)
y_test  = aggregate_generator_labels(data_generator=test_generator)

y_test_dec = binary_label_to_decimal(y_test)
print('Train size:',len(y_train),',',len(X_train_stacked))
print('Validation size:',len(y_valid),',',len(X_valid_stacked))
print('Test size:',len(y_test),',',len(X_test_stacked))
print('Stacked train data size:', X_train_stacked.shape[1])

In [ ]:
# Build ensemble model
optimizer_name = 'Adam'
lr = 0.0001
optimizer = Adam(learning_rate=lr,beta_1=0.9,beta_2=0.999,epsilon=1e-07)
model_stacked = build_stacked_model(num_layer,len(members),num_classes,optimizer)
plot_model(model_stacked, to_file= path_out+'stacked_model.png')
print(model_stacked.summary())

# Train ensemble model

In [ ]:
# Train the stacked model
epochs = 500
batch_size = 16

# Early stopping
monitor = 'val_loss'
min_delta = 0.0001
patience = 5
early_stopping_loss = EarlyStopping(monitor=monitor,min_delta=min_delta,patience=patience,\
                                    verbose=0,mode='auto',baseline=None,restore_best_weights=True)
model_stacked,history = fit_stacked_model(model_stacked, \
                                           X_train_stacked,\
                                           y_train,X_valid_stacked,\
                                           y_valid,\
                                           epochs,\
                                           batch_size,\
                                           early_stopping_loss)
plot_EM_trian_history(history,path_out,IFPLOT=True)

In [ ]:
# Save the trained stacked model
stacked_model_name = path_out+'stacked_model.h5'
model_stacked.save(stacked_model_name)

In [ ]:
# Write stacked model training hyperparameters
hp_filename = path_out + "stacked_model_hp_details.txt"
with open(hp_filename,"w") as f:
    f.write('Early Stopping: %s (min_delta = %.8f, patience = %i) \n' % (monitor, min_delta, patience))
    f.write('Optimizer: %s (lr = %f) \n' % (optimizer_name, lr))
    f.write('Training epoch: %i \n' % (epochs))
    f.write('Batch size: %i \n' % (batch_size))
    f.write('Train size: %i \n' % (len(y_train)))

# Compute test accuracy

In [ ]:
# Get the predictions on test set
y_pred_prob_stacked = model_stacked.predict(X_test_stacked) # predict
y_pred_stacked = binary_label_to_decimal(prediction_standardized(y_pred_prob_stacked)) #compute ensemble model accuracy

y_pred_prob = []
y_pred = []
for i in range(0,len(members)):
    y_pred_prob_i = evaluate_model(members[i], evaluation_generator=test_generator, workers=6)
    y_pred_i = binary_label_to_decimal(prediction_standardized(y_pred_prob_i))
    y_pred_prob.append(y_pred_prob_i)
    y_pred.append(y_pred_i)

In [ ]:
# Compute stacked model test accuracy
y_test_dec = binary_label_to_decimal(y_test)
stacked_acc_test = accuracy_score(y_test_dec,y_pred_stacked)
acc_test = []
for i in range(0,len(members)):
    acc_test_i = accuracy_score(y_test_dec,y_pred[i])
    acc_test.append(acc_test_i)
    
# Compare test accuracies
mean = np.mean(acc_test)
stdev = np.std(acc_test)
print('TEST ACCURACY:')
print('Ensemble model:',stacked_acc_test)
print('Individual models: %f (%f)' % (mean, stdev))
print('Individual accuracies:', acc_test)

In [ ]:
# Save test accuracies
acc_filename = path_out + "stacked_accuracy_summary.txt"
with open(acc_filename,"w") as f:
    f.write('Ensemble: %.8f \n' % (stacked_acc_test))
    f.write('Individual models: %f (%f) \n' % (mean, stdev))
    f.write('Individual accuracies: {}'.format(acc_test))

# Estimate uncertainty

In [ ]:
def brier_multiclass(y, y_pred_prob):
    return np.mean(np.sum((y-y_pred_prob)**2, axis=1))

In [ ]:
def top5(y_decimal,y_pred_prob):
    correct = 0
    for i in range(len(y_decimal)):
        y_true = y_decimal[i]
        y_pred = y_pred_prob[i]
        top5_idx = sorted(range(len(y_pred)), key=lambda i: y_pred[i])[-5:]
        if y_true in top5_idx:
            correct = correct+1
    top5_err = 1-correct/len(y_decimal)
    return top5_err

In [ ]:
# Estimate on test set
NLL =[]
brier = []
top_5 = []
MSE = []
for i in range(0,len(members)):
    NLL_i = log_loss(y_test_dec,y_pred_prob[i]) 
    brier_i = brier_multiclass(y_test, y_pred_prob[i])
    top_5_i = top5(y_test_dec,y_pred_prob[i])    
    MSE_i = mean_squared_error(y_test,y_pred_prob[i]) # ensemble 
    
    NLL.append(NLL_i)
    brier.append(brier_i)
    top_5.append(top_5_i)
    MSE.append(MSE_i)

NLL_mean = np.mean(NLL)
brier_mean = np.mean(brier)
MSE_mean = np.mean(MSE)
top_5_mean = np.mean(top_5)
print('NLL MEAN:',NLL_mean)
print('NLL:',NLL)
print('Brier MEAN:',brier_mean)
print('Brier:',brier)
print('MSE MEAN:',MSE_mean)
print('MSE:',MSE)
print('Top_5:',top_5)

In [ ]:
# Save uncertainty in text file
uncertainty_filename = path_out + "ensemble_uncertainty_BM_"+ str(len(members))+".txt"
with open(uncertainty_filename,"w") as f:
    f.write('NLL MEAN: %.8f \n' % NLL_mean)
    f.write('NLL: {}\n'.format(NLL))
    f.write('Brier MEAN: %.8f \n'% brier_mean)
    f.write('Brier: {}\n'.format(brier))
    f.write('MSE MEAN: %.8f \n'% MSE_mean)
    f.write('MSE: {}\n'.format(MSE))
    f.write('Top1 MEAN: %.8f \n' % (mean))
    f.write('Top1: {}\n'.format(acc_test))
    f.write('Top5 MEAN: %.8f \n' % (top_5_mean))
    f.write('Top5: {}\n'.format(top_5))
    

# Evaluate detailed performance improvement

In [ ]:
label_list = [int(i)-1 for i in char_list]

## Confusion matrix

In [ ]:
# Confusion matrix
CM_stacked = confusion_matrix(y_test_dec, y_pred_stacked, labels=label_list)
CM = []
for i in range(0,len(members)):
    CM_model_i = confusion_matrix(y_test_dec, y_pred[i], labels=label_list)
    CM.append(CM_model_i)

In [ ]:
def plot_hm(data,title,cbar=False ):
    v = np.linspace(0, 100, 6, endpoint=True)
    sns.heatmap(data,cmap="YlGnBu",xticklabels=list(data.head()),yticklabels=list(data.index),
                cbar=cbar,cbar_kws={'ticks':v},vmin=0, vmax=1)
    plt.xlabel(' ',fontsize=30,fontname='Calibri',color='black')
    plt.ylabel(' ',fontsize=30,fontname='Calibri',color='black')
    plt.xticks(rotation='horizontal',fontsize=20,fontname='Calibri',color='black')
    plt.yticks(rotation='horizontal',fontsize=20,fontname='Calibri',color='black')

In [ ]:
# Plot confusion matrix
df_cm_stacked = pd.DataFrame(CM_stacked,index = label_list,columns = label_list)
df_cm = []
for i in range(0,len(members)):
    df_cm_i = pd.DataFrame(CM[i],index = label_list,columns = label_list)
    df_cm.append(df_cm_i)

v = np.linspace(0, 100, 6, endpoint=True)
n_col = math.ceil((len(members)+1)/2)
fig, axs = plt.subplots(2, 3,figsize=(15, 10))
fig.suptitle('Confusion Matrix')
for i in range(0,len(members)+1):
    row = i//n_col
    col = i%n_col
    if i == 0:
        sns.heatmap(ax=axs[row,col],data=df_cm_stacked, annot=True,cmap="YlGnBu",
                  cbar=False,cbar_kws={'ticks':v},vmin=0, vmax=100)
        axs[row,col].set_title('Stacked model',fontsize=20,fontname='Calibri')
    else:
        sns.heatmap(ax=axs[row,col],data=df_cm[i-1], annot=True,cmap="YlGnBu",
                  cbar=False,cbar_kws={'ticks':v},vmin=0, vmax=100)
        axs[row,col].set_title('Base model '+str(i),fontsize=20,fontname='Calibri')       
CM_figname = path_out+'CM.png'
plt.savefig(CM_figname)

## Accuracy by class

In [ ]:
acc_by_class_stacked = CM_stacked.diagonal()/CM_stacked.sum(axis=1)
acc_by_class = [acc_by_class_stacked]
col_names = ['Stacked']
for i in range(0,len(members)):
    acc_by_class_model_i = CM[i].diagonal()/CM[i].sum(axis=1)
    acc_by_class.append(acc_by_class_model_i)
    col_names.append('Model'+str(i+1))

acc_by_class_mean  = np.mean(acc_by_class,axis = 0)
acc_by_class_stdev = np.std(acc_by_class,axis = 0)
acc_by_class.append(acc_by_class_mean)
acc_by_class.append(acc_by_class_stdev)
col_names.append('MEAN')
col_names.append('STDEV')

print('ACCURACY BY CLASS')
acc_by_class = pd.DataFrame(acc_by_class,index=col_names,columns=label_list)
print(acc_by_class)
acc_by_class.to_csv(path_out+'stacked_accuracy_by_class.csv')

## ROC

In [ ]:
fpr_stacked,tpr_stacked,roc_auc_stacked = compute_roc(label_list,y_test,y_pred_prob_stacked)
fpr = []
tpr = []
roc_auc = []
for i in range(len(members)):
    fpr_i,tpr_i,roc_auc_i = compute_roc(label_list,y_test,y_pred_prob[i])
    fpr.append(fpr_i)
    tpr.append(tpr_i)
    roc_auc.append(roc_auc_i)

# Plot
n_col = math.ceil((len(members)+1)/2)   
fig, axs = plt.subplots(2, 3,figsize=(15, 10)) # change 3 to n_col for more BMs
fig.suptitle('ROC')
for i in range(len(members)+1):
    row = i//n_col
    col = i%n_col
    if i == 0:
        plot_roc(axs[row,col],label_list,fpr_stacked,tpr_stacked,roc_auc_stacked,'Stacked')
    elif i == len(members):
        plot_roc(axs[row,col],label_list,fpr[i-1],tpr[i-1],roc_auc[i-1],'Base Model '+str(i))
    else:
        plot_roc(axs[row,col],label_list,fpr[i-1],tpr[i-1],roc_auc[i-1],'Base Model '+str(i))
plt.tight_layout()

ROC_figname = path_out+'ROC.png'
plt.savefig(ROC_figname)
plt.show()

In [ ]:
ave_tpr_stacked,std_tpr_stacked,ave_roc_auc_stacked = ave_roc(label_list,fpr_stacked,tpr_stacked,roc_auc_stacked)
ave_tpr = []
std_tpr = []
ave_roc_auc = []
for i in range(len(members)):
    ave_tpr_i,std_tpr_i,ave_roc_auc_i = ave_roc(label_list,fpr[i],tpr[i],roc_auc[i])
    ave_tpr.append(ave_tpr_i)
    std_tpr.append(std_tpr_i)
    ave_roc_auc.append(ave_roc_auc_i)

# Plot
n_col = math.ceil((len(members)+1)/2)   
fig, axs = plt.subplots(2, 3,figsize=(15, 10))
fig.suptitle('ROC')
for i in range(len(members)+1):
    row = i//n_col
    col = i%n_col
    if i == 0:
        plot_ave_roc(axs[row,col],ave_tpr_stacked,std_tpr_stacked,ave_roc_auc_stacked,'Stacked')
    else:
        plot_ave_roc(axs[row,col],ave_tpr[i-1],std_tpr[i-1],ave_roc_auc[i-1],'Model '+str(i))
plt.tight_layout()

print('ROC AUC:', ave_roc_auc)

AVE_ROC_figname = path_out+'AVE_ROC.png'
plt.savefig(AVE_ROC_figname)

ave_roc_filename = path_out + "AVE_ROC.txt"
with open(ave_roc_filename,"w") as f:
    f.write('Individual accuracies: {}'.format(ave_roc_auc))

%matplotlib inline
import seaborn as sns
cmap = sns.color_palette("tab10")

fig, ax = plt.subplots(figsize=(5,5))
for i in range(len(members)+1):
    if i==0:
        ax.plot(fpr,ave_tpr_stacked,label='stacked,AUC = '+str(ave_roc_auc_stacked),linewidth=4,c=cmap[i])
    else:
        ax.plot(fpr,ave_tpr[i-1],label='Model '+str(i) + ',AUC = '+str(ave_roc_auc[i-1]),linewidth=4,ls = '--',c=cmap[i])

plt.xticks(fontsize=20,fontname='Calibri', color="black")
plt.yticks(fontsize=20,fontname='Calibri', color="black")
plt.xlabel('FPR',fontsize=30,fontname='Calibri',color='black',y=-1)
plt.ylabel('TPR',fontsize=30,fontname='Calibri',color='black')
plt.legend(loc= 'lower center',bbox_to_anchor=(1.8, 0.5),fontsize=10,frameon=False)
plt.show()

In [ ]:
fpr = np.linspace(0, 1, 201)
cmap = sns.color_palette("Blues",10)
rand_i = shuffle(np.linspace(3,8,6))
cmap2 = sns.color_palette("Paired")
fig, ax = plt.subplots(figsize=(5,5))

for i in range(len(members)+1):
    if i ==len(members):
        ax.plot(fpr,ave_tpr_stacked,label='Stacked  ,AUC = '+str(ave_roc_auc_stacked),linewidth=6,c=cmap2[7])
    else:
        ax.plot(fpr,ave_tpr[i],label=str(i)+'             ,AUC = '+str(ave_roc_auc[i]),linewidth=6,c=cmap[int(rand_i[i])],linestyle='--')

ax.set_xlim([0.0, 1.05])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate',fontsize=20,fontname='Calibri', color="black")
ax.set_ylabel('True Positive Rate',fontsize=20,fontname='Calibri', color="black")
ax.tick_params(axis='x', labelsize=20,color="black",labelcolor="black") 
ax.tick_params(axis='y', labelsize=20,color="black",labelcolor="black")  
ax.set_xticks(np.round(np.linspace(0,1,6),2))
ax.set_yticks(np.round(np.linspace(0,1,6),2))
plt.legend(loc="right",bbox_to_anchor=(2, 0.5),frameon=False)
ave_roc_comparison_figname = path_out+'AVE_ROC_COMPARISON.png'
plt.savefig(ave_roc_comparison_figname)